In [1]:
#this section contains all the commands that need to be run once at the begining of the script
import pandas as pd
import numpy as np
import os

path = r'C:\Users\sryan\Documents\CareerFoundry\Insta Cart Basket Analysis'

ords_prod_merge = pd.read_pickle(os.path.join(path, '02 data', 'Prepared data', 'orders_products_new_col.pkl'))
#df_prod = pd.read_csv(os.path.join(path, '02 data', 'Original data', 'products.csv'), index_col = False)
#df_ords_prior = pd.read_csv(os.path.join(path, '02 data', 'Original data', 'orders_products_prior.csv'), index_col = False)

# Aggregate data and get descriptive statistics

In [3]:
#Get descriptive stats from the entire dataframe
ords_prod_merge.groupby('department_id').agg({'Cust_order_num': ['mean']})

,Cust_order_num
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


all of the available product id's are now represented
the mean value for those in the subset are different than the values from the complete data set
the code also took a little longer to run

In [3]:
df.groupby('department_id').agg({'Cust_order_num': ['mean', 'min', 'max']})

NameError: name 'df' is not defined

# flag customers based on number of orders

In [2]:
#create a column of max orders for each customer id
ords_prod_merge['max_order'] = ords_prod_merge.groupby(['user_id'])['Cust_order_num'].transform(np.max)

In [3]:
#assign a flag for each customer based on the number of orders
ords_prod_merge.loc[ords_prod_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prod_merge.loc[(ords_prod_merge['max_order'] <= 40) & (ords_prod_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prod_merge.loc[ords_prod_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [4]:
ords_prod_merge['loyalty_flag'].value_counts(dropna=False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [5]:
ords_prod_merge.head()

,order_id,user_id,Cust_order_num,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0,...,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,hour_busyness,max_order,loyalty_flag
0,2539329,1,1,2,8,NaN,196,1,0,195,...,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer
1,2398795,1,2,3,7,15.0,196,1,1,195,...,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer
2,473747,1,3,3,12,21.0,196,1,1,195,...,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer
3,2254736,1,4,4,7,29.0,196,1,1,195,...,77,7,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer
4,431534,1,5,4,15,28.0,196,1,1,195,...,77,7,9.0,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer


In [6]:
#Get descriptive stats based on loyalty
ords_prod_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

# flag customers based on purchase prices

In [7]:
#create a column of mean prices
ords_prod_merge['mean_price'] = ords_prod_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [ ]:
ords_prod_merge[['user_id', 'prices', 'mean_price']].head(60)

In [8]:
#assign a flag for each customer based on the mean prices
ords_prod_merge.loc[ords_prod_merge['mean_price'] < 10, 'spend_flag'] = 'Low Spender'
ords_prod_merge.loc[ords_prod_merge['mean_price'] >= 10, 'spend_flag'] = 'High Spender'

In [ ]:
ords_prod_merge.head(50)

# flag customers based on days since last order

In [9]:
#create a column of median days since last order
ords_prod_merge['median_days_since_last'] = ords_prod_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [ ]:
ords_prod_merge[['user_id', 'median_days_since_last', 'days_since_prior_order']].head(60)

In [10]:
#assign a flag for each customer based on the median days since last order
ords_prod_merge.loc[ords_prod_merge['median_days_since_last'] > 20, 'frequency_flag'] = 'Non-frequent customer'
ords_prod_merge.loc[(ords_prod_merge['median_days_since_last'] > 10) & (ords_prod_merge['median_days_since_last'] <= 20), 'frequency_flag'] = 'Regular customer'
ords_prod_merge.loc[ords_prod_merge['median_days_since_last'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [11]:
ords_prod_merge['frequency_flag'].value_counts()

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
Name: frequency_flag, dtype: int64

In [12]:
#export data to pkl
ords_prod_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_flagged.pkl'))